In [1]:
! pip install mlflow boto3 awscli optuna xgboost imbalanced-learn
! pip install optuna

  Using cached docutils-0.19-py3-none-any.whl.metadata (2.7 kB)
  Using cached rsa-4.7.2-py3-none-any.whl.metadata (3.6 kB)
   ---------------------------------------- 0.0/4.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.6 MB 660.6 kB/s eta 0:00:07
    --------------------------------------- 0.1/4.6 MB 787.7 kB/s eta 0:00:06
   --- ------------------------------------ 0.5/4.6 MB 3.5 MB/s eta 0:00:02
   ------------------- -------------------- 2.2/4.6 MB 12.8 MB/s eta 0:00:01
   ---------------------------------------  4.6/4.6 MB 21.1 MB/s eta 0:00:01
   ---------------------------------------- 4.6/4.6 MB 18.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   - -------------------------------------- 2.3/72.0 MB 48.6 MB/s eta 0:00:02
   -- ------------------------------------- 3.9/72.0 MB 42.2 MB/s eta 0:00:02
   --- ------------------------------------ 6.2/7


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import mlflow
# step 1 : Set up the mlflow tracking server 
mlflow.set_tracking_uri("http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/")

In [3]:
# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

2025/11/22 19:04:47 INFO mlflow.tracking.fluent: Experiment with name 'Exp 5 - ML Algos with HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://youtube-recommender-exp-bucket/5', creation_time=1763818486726, experiment_id='5', last_update_time=1763818486726, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [4]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split,StratifiedKFold,cross_val_score
from sklearn.metrics import accuracy_score,classification_report,f1_score,make_scorer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

c:\Users\mukti\Desktop\youtube_sentiment_analysis\youtube_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.read_csv(r'C:\Users\mukti\Desktop\youtube_sentiment_analysis\youtube_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [6]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for KNN regression

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

## Step 2: Train-test split BEFORE sampling

X_train_raw,X_test_raw,y_train,y_test = train_test_split(
        df['clean_comment'],df['category'],
        test_size=0.2,
        random_state=42,
        stratify=df['category']
)

# Step 3: BOW vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000

vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=max_features)

X_train_vec = vectorizer.fit_transform(X_train_raw)
X_test_vec = vectorizer.transform(X_test_raw)

# Step 4: Apply Random Undersampler to train only
rus = RandomUnderSampler(random_state=42)
X_train_res, y_train_res = rus.fit_resample(X_train_vec, y_train)



#Function to log results in MLflow
def log_mlflow(model_name,model,X_train,X_test,y_train,y_test):
    with mlflow.start_run():
        #log the model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_Undersampling_BOW_Trigram")
        mlflow.set_tag("experiment_type", "Algorithm comparison")

        #log algorith name as parameter
        mlflow.log_param("algo_name",model_name)

        #Train model
        model.fit(X_train,y_train)
        y_pred =model.predict(X_test)

        # log accuracy
        accuracy = accuracy_score(y_test,y_pred)
        mlflow.log_metric("accuracy",accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for KNN
def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors',3,30)
    p = trial.suggest_categorical('p', [1, 2])

    model = KNeighborsClassifier(n_neighbors=n_neighbors,p=p)
    model.fit(X_train_res, y_train_res)
    preds = model.predict(X_test_vec)

    return accuracy_score(y_test,preds)


# Step 7: Run Optuna for KNN, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_knn, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'], p=best_params['p'])

    # Log the best model with MLflow, passing the algo_name as "KNN"
    log_mlflow("KNN", best_model, X_train_res, X_test_vec, y_train_res, y_test)

# Run the experiment for KNN
run_optuna_experiment()




[I 2025-11-22 19:04:55,910] A new study created in memory with name: no-name-ae8a92d7-7d34-404f-ab2d-f6d1dc6c7d32
[I 2025-11-22 19:05:00,220] Trial 0 finished with value: 0.4337924451111414 and parameters: {'n_neighbors': 7, 'p': 2}. Best is trial 0 with value: 0.4337924451111414.
[I 2025-11-22 19:05:07,875] Trial 1 finished with value: 0.47429428610391383 and parameters: {'n_neighbors': 9, 'p': 1}. Best is trial 1 with value: 0.47429428610391383.
[I 2025-11-22 19:05:12,946] Trial 2 finished with value: 0.45738442656484385 and parameters: {'n_neighbors': 13, 'p': 1}. Best is trial 1 with value: 0.47429428610391383.
[I 2025-11-22 19:05:15,476] Trial 3 finished with value: 0.4294286103913814 and parameters: {'n_neighbors': 8, 'p': 2}. Best is trial 1 with value: 0.47429428610391383.
[I 2025-11-22 19:05:20,377] Trial 4 finished with value: 0.45738442656484385 and parameters: {'n_neighbors': 13, 'p': 1}. Best is trial 1 with value: 0.47429428610391383.
[I 2025-11-22 19:05:23,474] Trial 5 f

🏃 View run KNN_Undersampling_BOW_Trigram at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/5/runs/8904a62576ac4bd8ab98c459a2897fae
🧪 View experiment at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/5
